In [1]:
from db_info import user, password, host, port, database
import pandas as pd
import mysql.connector

db_connection = mysql.connector.connect(
user = user,
password = password,
host = host,
port = port,
database = database,
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_content")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])

In [2]:
cursor.execute("select * from injury_list il inner join injury_content ic on il.판례일련번호=ic.판례정보일련번호")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   판례일련번호    4189 non-null   int64 
 1   사건명       4188 non-null   object
 2   사건번호      4189 non-null   object
 3   선고일자      4189 non-null   object
 4   법원명       4189 non-null   object
 5   사건종류명     4189 non-null   object
 6   판례상세링크    4189 non-null   object
 7   판례정보일련번호  4189 non-null   int64 
 8   사건번호      4189 non-null   object
 9   선고        4189 non-null   object
 10  판결유형      4189 non-null   object
 11  판시사항      3417 non-null   object
 12  판결요지      3028 non-null   object
 13  참조조문      3392 non-null   object
 14  참조판례      1925 non-null   object
 15  판례내용      4189 non-null   object
dtypes: int64(2), object(14)
memory usage: 523.8+ KB


In [13]:
df['판례내용'].iloc[1944]=df['판례내용'].iloc[1944].replace('】','】')

C:\Users\user\AppData\Local\Temp\ipykernel_8988\408891709.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['판례내용'].iloc[1944]=df['판례내용'].iloc[1944].replace('】','】')
C:\Users\user\AppData\Local\Temp\ipykernel_8988\408891709.py:1: Settin

In [4]:
import re
import pandas as pd

df_split = df.copy()
# 판례본문 내용 분리 - 이유 기준 분리

# '【이' 앞부분과 '【이'를 포함한 뒷부분으로 나누는 코드
df_split['판례내용_상단'] = df['판례내용'].str.split('【이').str[0]
df_split['판례내용_이유'] = df['판례내용'].str.extract(r'(【이.*)')

In [5]:
df_split[df_split['판례내용_이유'].isnull()]

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,판례상세링크,판례정보일련번호,사건번호,선고,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유
202,215165,보험금등청구의소[국민건강보험법에 따른 보험급여를 받은 피해자의 가해자에 대한 치료비...,2018다287935,2021.03.18,대법원,민사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,215165,2018다287935,선고,전원합의체 판결,국민건강보험공단이 불법행위의 피해자에게 보험급여를 한 다음 국민건강보험법 제58조 ...,[다수의견] 국민건강보험공단(이하 ‘공단’이라고 한다)의 손해배상청구권 대위를 인정...,"국민건강보험법 제1조, 제53조 제1항, 제57조, 제58조, 사회보장기본법 제3조...","대법원 2002. 1. 8. 선고 2001다40022, 40039 판결(변경), 대...","[원고, 피상고인 겸 상고인] 원고 (소송대리인 변호사 박기억)<br/>[피고, ...","[원고, 피상고인 겸 상고인] 원고 (소송대리인 변호사 박기억)<br/>[피고, ...",NaN
647,214247,특수상해미수ㆍ폭행[임의적 감경 사건],2018도5475,2021.01.21,대법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,214247,2018도5475,선고,전원합의체 판결,‘임의적 감경’의 의미 / 임의적 감경사유의 존재가 인정되고 법관이 그에 따라 징역...,[다수의견] 필요적 감경의 경우에는 감경사유의 존재가 인정되면 반드시 형법 제55조...,"헌법 제12조 제1항, 형법 제1조 제1항, 제25조 제2항, 제55조 제1항 제3...","대법원 1991. 6. 11. 선고 91도985 판결(공1991, 1970), 대법...",[피 고 인] 피고인<br/>[상 고 인] 피고인<br/>[원심판결] 의정부지법 2...,[피 고 인] 피고인<br/>[상 고 인] 피고인<br/>[원심판결] 의정부지법 2...,NaN
1765,223577,"살인,살인미수,현주건조물방화치상(인정된 죄명：현주건조물방화)",98고합1266,1999.04.20,서울지방법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,223577,98고합1266,선고,판결：항소기각·상고기각,[1] 유아의 증언능력의 판단 기준<br/> [2] 사건 당시 만 4세 6개월...,[1] 우리 형사소송법은 법원은 원칙적으로 누구든지 증인으로 신문할 수 있다고 ...,"[1] 형사소송법 제146조, /[2] 형사소송법 제...",[1][2] <br/>대법원 1991. 5. 10. 선고 91도579 판결(공199...,【피 고 인】 피고인<br/>【변 호 인】 변호사 김경천<br/>【주 문】<...,【피 고 인】 피고인<br/>【변 호 인】 변호사 김경천<br/>【주 문】<...,NaN
2286,223551,자동차운전면허취소처분등취소,88누46,1988.04.12,대법원,일반행정,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,223551,88누46,선고,판결,가. 운전면허처분취소에 있어서의 재량권행사의 기준<br/>나. 음주운전중 요치 3주...,가. 운전면허를 받은 사람이 음주운전을 하다가 고의 또는 과실로 교통사고를 일으킨 ...,"도로교통법 제78조 제4호, 도로교통법시행규칙 제53조...",None,"【원고, 피상고인】 원고<br/>【피고, 상고인】 서울특별시장<br/>【원심판결...","【원고, 피상고인】 원고<br/>【피고, 상고인】 서울특별시장<br/>【원심판결...",NaN
4137,214863,"상습폭행(인정된 죄명 폭행), 아동복지법위반(상습아동학대){인정된 죄명 아동복지법위...",2018노1659,2020.02.13,서울서부지방법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,214863,2018노1659,선고,판결,\n,\n,\n,\n,[피 고 인] 피고인<br/>[항 소 인] 쌍방<br/>[검 사]...,[피 고 인] 피고인<br/>[항 소 인] 쌍방<br/>[검 사]...,NaN


In [6]:
df_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   판례일련번호    4189 non-null   int64 
 1   사건명       4188 non-null   object
 2   사건번호      4189 non-null   object
 3   선고일자      4189 non-null   object
 4   법원명       4189 non-null   object
 5   사건종류명     4189 non-null   object
 6   판례상세링크    4189 non-null   object
 7   판례정보일련번호  4189 non-null   int64 
 8   사건번호      4189 non-null   object
 9   선고        4189 non-null   object
 10  판결유형      4189 non-null   object
 11  판시사항      3417 non-null   object
 12  판결요지      3028 non-null   object
 13  참조조문      3392 non-null   object
 14  참조판례      1925 non-null   object
 15  판례내용      4189 non-null   object
 16  판례내용_상단   4189 non-null   object
 17  판례내용_이유   4184 non-null   object
dtypes: int64(2), object(16)
memory usage: 589.2+ KB


In [7]:
df_split.to_csv('data/hurtcombined.csv')

In [16]:
import re
import pandas as pd

df_split = df.copy()
# 판례본문 내용 분리 - 이유 기준 분리

# '【이' 앞부분과 '【이'를 포함한 뒷부분으로 나누는 코드
df_split['판례내용_상단'] = df['판례내용'].str.split('【이').str[0]
df_split['판례내용_이유'] = df['판례내용'].str.extract(r'(【이.*)')

# 판례내용 상단에서 재판당사자 추출

# '문단의 첫 번째 '【'부터 '【주' 앞까지의 텍스트를 포함하여 추출
def extract_text_with_first_bracket_to_ju(content):
    # 정규식으로 첫 번째 '【'부터 '【주'까지 텍스트 추출 (첫 번째 '【' 포함)
    match = re.search(r'(【.*?【\s*주)', content, re.DOTALL)
    if match:
        return match.group(1).strip()  # 추출한 텍스트의 양쪽 공백 제거
    return None  # 매치되지 않으면 None 반환

# '재판당사자' 열 생성 (첫 번째 '【'부터 '【주'까지 추출, 첫 번째 '【' 포함)
df_split['재판당사자'] = df_split['판례내용_상단'].apply(extract_text_with_first_bracket_to_ju)

# 결과 확인
#df_service_split.head()

def extract_and_clean_text(content):
    if isinstance(content, str):  # 내용이 문자열인지 확인
        matches = re.findall(r'【(.*?)】', content)  # '【'와 '】' 사이의 모든 텍스트 찾기
        cleaned_texts = [match.replace(' ', '') for match in matches]  # 공백 제거
        return cleaned_texts  # 결과 리스트 반환
    return []  # 문자열이 아닌 경우 빈 리스트 반환

# 결과 열 생성
df_split['재판당사자_텍스트'] = df_split['재판당사자'].apply(extract_and_clean_text)

# 결과 확인
df_split[['재판당사자_텍스트']]

# 판례내용 이유 부분의 대괄호를 삭제하여 본문으로 변환


service_reason = []

for idx in range(len(df_split)):
    content = df_split['판례내용_이유'].iloc[idx]
    
    # 문자열인지 확인 (문자열이 아니면 건너뜀)
    if isinstance(content, str):
        content_parts = re.split(r'【(.*?)】', content)
        reason_dict = {}

        for j in range(len(content_parts)):
            if content_parts[j].replace(' ', '') in ['이유']:
                cleaned_content = content_parts[j+1]
                cleaned_content = cleaned_content.replace('<br/>', '').replace('\r', '').replace('\n', '')
                cleaned_content = cleaned_content.replace('【', '').replace('】', '').strip()
                cleaned_content = re.sub(r'\s+', ' ', cleaned_content)
                reason_dict[content_parts[j].replace(' ', '')] = cleaned_content
        
        service_reason.append(reason_dict)
    else:
        # 비문자열 데이터 처리 (필요한 경우 빈 딕셔너리 또는 다른 기본값)
        service_reason.append({})

# 결과를 '판례내용_이유2' 열에 저장
df_split['판례내용_이유2'] = service_reason

df_split.info()

# 전처리 함수 정의
def preprocess_content(content):
    if pd.isna(content):  # NaN 값에 대한 예외 처리
        return content
    
    # content가 문자열인지 확인
    if isinstance(content, str):
        content = content.replace('<br/>', '')
        content = content.replace('\n', '')
        content = content.replace('\r', '')
    
    return content

# 여러 열에 전처리를 적용할 경우
columns_to_preprocess = ['참조조문','참조판례', '판례내용', '판례내용_상단', '판례내용_이유', '재판당사자']

# applymap이 아닌 각 열에 개별적으로 apply 사용
for col in columns_to_preprocess:
    df_split[col] = df_split[col].apply(preprocess_content)

# 전처리 후 데이터프레임 확인
df_split.head()

df2=df_split
df2.to_csv('data/hurtcombined.csv')

# 판례데이터를 읽은 뒤 목록과 본문을 합친 뒤 분할해서 전처리

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   판례정보일련번호   4189 non-null   int64 
 1   사건번호       4189 non-null   object
 2   선고         4189 non-null   object
 3   판결유형       4189 non-null   object
 4   판시사항       3417 non-null   object
 5   판결요지       3028 non-null   object
 6   참조조문       3392 non-null   object
 7   참조판례       1925 non-null   object
 8   판례내용       4189 non-null   object
 9   판례내용_상단    4189 non-null   object
 10  판례내용_이유    4184 non-null   object
 11  재판당사자      4173 non-null   object
 12  재판당사자_텍스트  4189 non-null   object
 13  판례내용_이유2   4189 non-null   object
dtypes: int64(1), object(13)
memory usage: 458.3+ KB


In [17]:
df_split['판례내용_이유2'].iloc[1150]

{}

In [6]:
import re
import pandas as pd

df_split = df.copy()
# 판례본문 내용 분리 - 이유 기준 분리

# '【이' 앞부분과 '【이'를 포함한 뒷부분으로 나누는 코드
df_split['판례내용_상단'] = df['판례내용'].str.split('【이').str[0]
df_split['판례내용_이유'] = df['판례내용'].str.extract(r'(【이.*)')

# 판례내용 상단에서 재판당사자 추출

# '문단의 첫 번째 '【'부터 '【주' 앞까지의 텍스트를 포함하여 추출
def extract_text_with_first_bracket_to_ju(content):
    # 정규식으로 첫 번째 '【'부터 '【주'까지 텍스트 추출 (첫 번째 '【' 포함)
    match = re.search(r'(【.*?【\s*주)', content, re.DOTALL)
    if match:
        return match.group(1).strip()  # 추출한 텍스트의 양쪽 공백 제거
    return None  # 매치되지 않으면 None 반환

# '재판당사자' 열 생성 (첫 번째 '【'부터 '【주'까지 추출, 첫 번째 '【' 포함)
df_split['재판당사자'] = df_split['판례내용_상단'].apply(extract_text_with_first_bracket_to_ju)

# 결과 확인
#df_service_split.head()

def extract_and_clean_text(content):
    if isinstance(content, str):  # 내용이 문자열인지 확인
        matches = re.findall(r'【(.*?)】', content)  # '【'와 '】' 사이의 모든 텍스트 찾기
        cleaned_texts = [match.replace(' ', '') for match in matches]  # 공백 제거
        return cleaned_texts  # 결과 리스트 반환
    return []  # 문자열이 아닌 경우 빈 리스트 반환

# 결과 열 생성
df_split['재판당사자_텍스트'] = df_split['재판당사자'].apply(extract_and_clean_text)

# 결과 확인
df_split[['재판당사자_텍스트']]

# 판례내용 이유 부분의 대괄호를 삭제하여 본문으로 변환


,재판당사자_텍스트
0,"[원고,상고인겸피상고인, 피고,피상고인, 피고,피상고인겸상고인, 환송판결]"
1,"[피고인, 상고인, 원심판결]"
2,"[피고인, 상고인, 변호인, 원심판결]"
3,"[피고인, 상고인, 변호인, 원심판결]"
4,"[피고인, 상고인, 변호인, 원심판결]"
...,...
4184,"[피고인, 항소인, 제1심]"
4185,"[상고인,피고인, 원심판결]"
4186,"[상고인,피고인, 변호인, 원심판결]"
4187,"[피고인,상고인, 원심]"


In [7]:
df_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189 entries, 0 to 4188
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   판례정보일련번호   4189 non-null   int64 
 1   사건번호       4189 non-null   object
 2   선고         4189 non-null   object
 3   판결유형       4189 non-null   object
 4   판시사항       3417 non-null   object
 5   판결요지       3028 non-null   object
 6   참조조문       3392 non-null   object
 7   참조판례       1925 non-null   object
 8   판례내용       4189 non-null   object
 9   판례내용_상단    4189 non-null   object
 10  판례내용_이유    4184 non-null   object
 11  재판당사자      4173 non-null   object
 12  재판당사자_텍스트  4189 non-null   object
dtypes: int64(1), object(12)
memory usage: 425.6+ KB


In [2]:
import pandas as pd
df2=pd.read_csv('data/hurtcombined.csv')

In [6]:
from transformers import BertModel, DistilBertModel, BertTokenizer
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

In [8]:
df2['판시사항'] = df2['판시사항'].fillna('')


In [9]:
from kobert_transformers import get_tokenizer
import torch

model = distilbert_model  # 모델 선언
tokenizer = get_tokenizer()  # 토크나이저 선언

# 각 문장을 토큰화, id화
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in df2['판시사항']]
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_sentences]

In [10]:
len(tokenid_list)


4189

In [12]:
# sequence를 같은 길이로 padding
max_length = max(len(ids) for ids in tokenid_list)
padded_tokenid_list = [ids + [0] * (max_length - len(ids)) for ids in tokenid_list]

In [13]:
# Longtensor로 dtype변경
tokenid_tensor = torch.LongTensor(padded_tokenid_list)


In [14]:
# 추론
tensor_output = model(tokenid_tensor)  # 모델에 위의 전처리 과정을 거친 자연어를 input으로 넣는다
print(tensor_output[0].shape)  # Output shape
print(tensor_output[0])  # Output 확인

RuntimeError: The size of tensor a (924) must match the size of tensor b (512) at non-singleton dimension 1

그냥 넣으면 내용이 너무 길어져서 안됐고 OKT 토큰화한 값을 넣었으나 이 또한 마찬가지로 메모리 부족이 발생한다.

In [15]:
def sliding_window_tokenize(tokens, window_size=256, stride=128):
    """
    긴 토큰 리스트를 슬라이딩 윈도우 방식으로 나누는 함수.
    
    :param tokens: 토큰화된 리스트
    :param window_size: 각 윈도우의 크기
    :param stride: 겹치는 부분의 길이
    :return: 슬라이딩 윈도우로 나눈 토큰 리스트들의 리스트
    """
    token_windows = []
    for start in range(0, len(tokens), stride):
        end = start + window_size
        token_window = tokens[start:end]
        if len(token_window) == window_size:  # 윈도우 크기에 맞는 경우에만 추가
            token_windows.append(token_window)
    return token_windows

# 예제: 긴 토큰화된 문장을 슬라이딩 윈도우 방식으로 나누기
tokenized_sentence = tokenizer.tokenize(df2['판시사항'][0])
token_windows = sliding_window_tokenize(tokenized_sentence, window_size=256, stride=128)

# 각 토큰 윈도우를 확인
for i, window in enumerate(token_windows):
    print(f"윈도우 {i + 1}: {window}")

윈도우 1: ['▁[', '1', ']', '▁국내', '사업', '장을', '▁가진', '▁외국', '법인', '이', '▁국내', '▁및', '▁국', '외', '에', '▁걸쳐', '▁선박', '에', '▁의한', '▁국제', '운', '송', '업', '을', '▁영', '위', '하며', '▁발생', '하는', '▁소득', '▁중', '▁국내에서', '▁승', '선', '한', '▁여', '객', '이나', '▁선', '적', '한', '▁화', '물', '에', '▁관련', '하여', '▁발생한', '▁소득', '만', '이', '▁구', '▁법인', '세', '법', '▁제', '93', '조', '▁제', '5', '호', '에', '▁따른', '▁해당', '▁외국', '법인', '의', '▁국내', '원', '천', '▁사업', '소득', '으로서', '▁법인', '세', '▁과세', '표', '준', '에', '▁포함', '되는', '지', '▁여부', '(', '적', '극', ')', '<', 'b', 'r', '/', '>', '▁[', '2', ']', '▁외국', '법인', '의', '▁국내', '원', '천', '소득', '▁총', '합', '계', '액', '에', '▁포함', '되는', '▁‘', '선', '박', '의', '▁외국', '항', '행', '소득', '’', '의', '▁존재', '▁및', '▁범위', '는', '▁원칙', '적으로', '▁과세', '관', '청', '이', '▁증명', '하여', '야', '▁하는', '지', '▁여부', '(', '적', '극', ')', '<', 'b', 'r', '/', '>', '▁[', '3', ']', '▁외국', '항', '행', '용', '역', '을', '▁제공하는', '▁외국', '법인', '에', '▁구', '▁부', '가', '가', '치', '세', '법', '령', '상', '▁영', '세', '율이', '▁적용', '되는', '▁과세', '표', '준',

In [16]:
token_ids_list = []
for tokens in token_windows:
    # [CLS]와 [SEP] 토큰 추가
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    # 토큰을 ID로 변환
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    # 패딩 추가 (길이를 max_length로 맞추기)
    token_ids = token_ids + [0] * (max_length - len(token_ids))
    token_ids_list.append(token_ids)

# 텐서로 변환
tokenid_tensor = torch.LongTensor(token_ids_list)

In [17]:
print(tokenid_tensor)

tensor([[   2,  702,   93,  363, 1138, 6503, 7187,  769, 3471, 6338, 7096, 1138,
         2184, 1132, 6995, 6896,  894, 2741, 6896, 3642, 1155, 7010, 6621, 6873,
         7088, 3376, 7044, 7810, 2243, 7794, 2828, 4257, 1139, 2949, 6559, 7828,
         3298, 5370, 7098, 2734, 7202, 7828, 5112, 6241, 6896, 1083, 7815, 2244,
         2828, 6150, 7096, 1115, 2327, 6579, 6335, 4128,  242, 7253, 4128,  172,
         7925, 6896, 1839, 5000, 3471, 6338, 7095, 1138, 7020, 7422, 2609, 6608,
         7080, 2327, 6579, 1060, 7741, 7288, 6896, 4864, 5889, 7318, 3310,   18,
         7202, 5543,   40,  253,  380,  435,   59,  257,  702,  119,  363, 3471,
         6338, 7095, 1138, 7020, 7422, 6608, 4512, 7842, 5436, 6835, 6896, 4864,
         5889,  712, 6559, 6280, 7095, 3471, 7846, 7881, 6608,  498, 7095, 4193,
         2184, 2319, 5760, 3543, 7203, 1060, 5474, 7431, 7096, 4293, 7815, 6844,
         4930, 7318, 3310,   18, 7202, 5543,   40,  253,  380,  435,   59,  257,
          702,  142,  363, 3

In [18]:
from kobert_transformers import get_tokenizer
import torch

def prepare_inputs_for_kobert(token_windows, max_length=256):
    """
    슬라이딩 윈도우로 나눈 토큰들을 KoBERT 입력 형식에 맞게 전처리하는 함수.
    
    :param token_windows: 슬라이딩 윈도우로 나눈 토큰 리스트
    :param max_length: 각 입력의 최대 길이 (윈도우 크기와 동일해야 함)
    :return: 전처리된 토큰 ID 텐서
    """
    token_ids_list = []
    for tokens in token_windows:
        # [CLS]와 [SEP] 토큰 추가
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        # 토큰을 ID로 변환
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        # 패딩 추가 (길이를 max_length로 맞추기)
        token_ids = token_ids + [0] * (max_length - len(token_ids))
        token_ids_list.append(token_ids)
    
    # 텐서로 변환
    tokenid_tensor = torch.LongTensor(token_ids_list)
    return tokenid_tensor

# 긴 문장을 슬라이딩 윈도우 방식으로 나누기
tokenized_sentence = tokenizer.tokenize("여기에 긴 문장을 입력하세요.")
token_windows = sliding_window_tokenize(tokenized_sentence, window_size=256, stride=128)

# 전처리하여 KoBERT에 입력할 텐서 준비
tokenid_tensor = prepare_inputs_for_kobert(token_windows, max_length=256)

# 텐서 출력 확인
print(tokenid_tensor)

tensor([], dtype=torch.int64)


In [22]:
print(df2['판시사항'].isnull().sum())  # NaN 개수 확인
print(df2['판시사항'].head())  # 처음 5개 데이터 확인
print(len(df2['판시사항']))  # 데이터프레임의 길이 확인

0
0     [1] 국내사업장을 가진 외국법인이 국내 및 국외에 걸쳐 선박에 의한 국제운송업을...
1      특수상해죄 및 특수협박죄의 구성요건 중 위험한 물건을 ‘휴대하여’의 의미 / 이...
2    형사소송법 제33조 제1항 제1호에서 필요적 국선변호인 선정사유 중 하나로 정한 ‘...
3      피고인이 별거 상태로 이혼소송 중이던 배우자 甲(女)이 거주하는 원룸에 베란다를...
4                                                     
Name: 판시사항, dtype: object
4189


In [23]:
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in df2['판시사항']]

# 토큰화된 내용 확인
print(tokenized_sentences[:5])  # 처음 5개 문장 출력

[['▁[', '1', ']', '▁국내', '사업', '장을', '▁가진', '▁외국', '법인', '이', '▁국내', '▁및', '▁국', '외', '에', '▁걸쳐', '▁선박', '에', '▁의한', '▁국제', '운', '송', '업', '을', '▁영', '위', '하며', '▁발생', '하는', '▁소득', '▁중', '▁국내에서', '▁승', '선', '한', '▁여', '객', '이나', '▁선', '적', '한', '▁화', '물', '에', '▁관련', '하여', '▁발생한', '▁소득', '만', '이', '▁구', '▁법인', '세', '법', '▁제', '93', '조', '▁제', '5', '호', '에', '▁따른', '▁해당', '▁외국', '법인', '의', '▁국내', '원', '천', '▁사업', '소득', '으로서', '▁법인', '세', '▁과세', '표', '준', '에', '▁포함', '되는', '지', '▁여부', '(', '적', '극', ')', '<', 'b', 'r', '/', '>', '▁[', '2', ']', '▁외국', '법인', '의', '▁국내', '원', '천', '소득', '▁총', '합', '계', '액', '에', '▁포함', '되는', '▁‘', '선', '박', '의', '▁외국', '항', '행', '소득', '’', '의', '▁존재', '▁및', '▁범위', '는', '▁원칙', '적으로', '▁과세', '관', '청', '이', '▁증명', '하여', '야', '▁하는', '지', '▁여부', '(', '적', '극', ')', '<', 'b', 'r', '/', '>', '▁[', '3', ']', '▁외국', '항', '행', '용', '역', '을', '▁제공하는', '▁외국', '법인', '에', '▁구', '▁부', '가', '가', '치', '세', '법', '령', '상', '▁영', '세', '율이', '▁적용', '되는', '▁과세', '표', '준', '을', 

In [24]:
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_sentences]

# ID 리스트 길이 확인
print([len(ids) for ids in tokenid_list][:5])  # 처음 5개 문장 길이 확인

# 최대 길이 설정
max_length = 512

# 패딩 처리
padded_tokenid_list = [ids[:max_length] + [0] * (max_length - len(ids[:max_length])) for ids in tokenid_list]

# 패딩된 리스트 길이 확인
print([len(ids) for ids in padded_tokenid_list][:5])  # 처음 5개 문장 길이 확인

[307, 118, 115, 199, 0]
[512, 512, 512, 512, 512]


In [25]:
# LongTensor로 변환
tokenid_tensor = torch.LongTensor(padded_tokenid_list)

# 텐서 크기 확인
print(tokenid_tensor.shape)  # 출력: (batch_size, max_length)

torch.Size([4189, 512])


In [26]:

# 추론
tensor_output = model(tokenid_tensor)  # 모델에 위의 전처리 과정을 거친 자연어를 input으로 넣는다
print(tensor_output[0].shape)  # Output shape
print(tensor_output[0])  # Output 확인

# 출력은 (batchsize, 토큰id의 길이, 768). 여기서 768은 임베딩 공간의 크기

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 52709818368 bytes.

In [32]:
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in okt_tokens]
print(tokenid_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# 다시 시도

In [10]:
import pandas as pd
#불용어 읽어오기
stopwords_file_path = 'stopwords_dh_lastlast.txt'
stopwords = []
with open(stopwords_file_path, 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file.readlines()]

df=pd.read_csv('data/hurtcombined.csv')
df=df.drop(df.columns[0],axis=1)
df.head()

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,판례상세링크,판례정보일련번호,사건번호.1,선고,...,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,재판당사자,재판당사자_텍스트,판례내용_이유2
0,241209,법인세등부과처분취소[국내사업장을 가진 외국법인이 국내 및 국외에 걸쳐 선박에 의한 ...,2022두51031,2024.07.25,대법원,세무,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,241209,2022두51031,선고,...,[1] 국내사업장을 가진 외국법인이 국내 및 국외에 걸쳐 선박에 의한 국제운송업을...,[1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것...,[1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것...,"[3] 대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2...","【원고, 상고인 겸 피상고인】 ○○○서비스 (소송대리인 법무법인 케이원챔버 담당변호...","【원고, 상고인 겸 피상고인】 ○○○서비스 (소송대리인 법무법인 케이원챔버 담당변호...",【이 유】 1. 원고의 원심 승소 부분에 대한 상고의 적법 여부에 관한 직권...,"【원고, 상고인 겸 피상고인】 ○○○서비스 (소송대리인 법무법인 케이원챔버 담당변호...","['원고,상고인겸피상고인', '피고,피상고인', '피고,피상고인겸상고인', '환송판결']",{'이유': '1. 원고의 원심 승소 부분에 대한 상고의 적법 여부에 관한 직권판단...
1,240925,폭행·특수상해(인정된죄명:상해)·특수협박(인정된죄명:협박)·감금[피고인이 위험한 물...,2023도18812,2024.06.13,대법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,240925,2023도18812,선고,...,특수상해죄 및 특수협박죄의 구성요건 중 위험한 물건을 ‘휴대하여’의 의미 / 이...,"형법 제258조의2 제1항, 제257조 제1항, 제284조, 제283조 제1항은 위...","형법 제257조 제1항, 제258조의2 제1항, 제283조 제1항, 제284조","대법원 2002. 6. 14. 선고 2002도1341 판결, 대법원 2004. 6....",【피 고 인】 피고인【상 고 인】 피고인 및 검사【원심판결】 대전지법 2023. 1...,【피 고 인】 피고인【상 고 인】 피고인 및 검사【원심판결】 대전지법 2023. 1...,【이 유】 상고이유를 판단한다. 1. 검사의 상고이유에 관하여 가. 2...,【피 고 인】 피고인【상 고 인】 피고인 및 검사【원심판결】 대전지법 2023. 1...,"['피고인', '상고인', '원심판결']",{'이유': '상고이유를 판단한다. 1. 검사의 상고이유에 관하여 가. 2019. ...
2,240809,상해[형사소송법 제33조 제1항 제1호가 정한 필요적 국선변호인 선정사유인 ‘구속’...,2021도6357,2024.05.23,대법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,240809,2021도6357,선고,...,형사소송법 제33조 제1항 제1호에서 필요적 국선변호인 선정사유 중 하나로 정한 ‘...,[다수의견] 형사소송법 제33조 제1항 제1호는 피고인에게 변호인이 없는 때에 ...,"헌법 제12조, 형사소송법 제33조 제1항 제1호, 제69조, 제473조 제1항, ...","대법원 2009. 5. 28. 선고 2009도579 판결(공2009하, 1060)(...",【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 유소정【원심판결】 인천...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 유소정【원심판결】 인천...,【이 유】 상고이유를 판단한다. 1. 사건의 개요와 쟁점 가. 소송의 진...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 유소정【원심판결】 인천...,"['피고인', '상고인', '변호인', '원심판결']",{'이유': '상고이유를 판단한다. 1. 사건의 개요와 쟁점 가. 소송의 진행 경과...
3,219875,주거침입·성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영)·상해,2021노802,2021.11.05,울산지방법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,219875,2021노802,선고,...,피고인이 별거 상태로 이혼소송 중이던 배우자 甲(女)이 거주하는 원룸에 베란다를...,피고인이 별거 상태로 이혼소송 중이던 배우자 甲(女)이 거주하는 원룸에 베란다를...,성폭력범죄의 처벌 등에 관한 특례법 제14조 제1항,NaN,【피 고 인】 피고인【항 소 인】 검사【검 사】 양재헌 외 1인【변 호 인】 ...,【피 고 인】 피고인【항 소 인】 검사【검 사】 양재헌 외 1인【변 호 인】 ...,【이 유】 1. 이 법원의 심판범위 가. 경합범으로 동시에 기소된 사건에 ...,【피 고 인】 피고인【항 소 인】 검사【검 사】 양재헌 외 1인【변 호 인】 ...,"['피고인', '항소인', '검사', '변호인', '원심판결']",{'이유': '1. 이 법원의 심판범위 가. 경합범으로 동시에 기소된 사건에 대하여...
4,214727,마약류관리에관한법률위반(향정)ㆍ마약류관리에관한법률위반(대마)ㆍ특수상해,2019고단4056,2020.02.19,부산지방법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,214727,"2019고단4056, 5160(병합), 6200(병합), 6427(병합)",선고,...,NaN,NaN,NaN,NaN,"【피 고 인】 피고인 1 외 2인【검 사】 김미선, 임홍석(기소), ...","【피 고 인】 피고인 1 외 2인【검 사】 김미선, 임홍석(기소), ...",【이 유】범 죄 사 실[범죄전력]피고인 1(2020. 6. 19.자 항소심 판...,"【피 고 인】 피고인 1 외 2인【검 사】 김미선, 임홍석(기소), ...","['피고인', '검사', '변호인']",{'이유': '범 죄 사 실[범죄전력]피고인 1(2020. 6. 19.자 항소심 판...


In [11]:
reason=df['판례내용_이유']

#NaN값 처리
reason= reason.fillna('')

In [22]:
length=[]
len(reason)
for i in range(len(reason)):
    length.append(len(reason[i]))
average=sum(length)/len(length)
print(average)
print(max(length))
print(min(length))
# 0이 아닌 최소 길이 찾기
min_length = min([l for l in length if l > 0])  # 0보다 큰 길이들 중에서 최소값 찾기
print(min_length)
# 내림차순으로 정렬
sorted_length = sorted(length, reverse=True)

print("내림차순 정렬된 리스트:", sorted_length)

3144.1375029840056
291722
0
7
내림차순 정렬된 리스트: [291722, 207029, 205967, 183159, 155715, 141313, 121655, 120632, 109196, 104686, 104583, 101636, 86550, 83133, 81889, 81526, 75409, 70841, 68434, 67227, 65344, 64892, 61146, 58855, 57873, 56061, 54097, 52816, 47870, 46366, 44727, 42480, 42339, 42246, 41647, 41346, 41125, 40138, 39752, 39677, 38307, 36912, 36684, 35846, 35749, 35189, 35165, 33762, 33737, 32474, 31353, 31098, 30685, 30466, 30179, 29802, 29770, 29605, 29430, 28934, 28804, 28463, 28368, 28248, 27932, 27215, 27087, 26933, 26628, 25586, 25024, 24857, 24281, 24050, 23941, 23836, 23604, 23395, 23019, 22915, 22820, 22791, 22406, 22183, 22117, 21640, 21302, 20896, 20873, 20762, 20634, 20609, 20561, 20461, 20157, 19941, 19931, 19463, 19087, 19032, 18938, 18680, 18635, 18510, 18377, 18349, 18166, 18025, 17932, 17789, 17766, 17707, 17229, 17228, 17139, 16822, 16745, 16738, 16624, 16538, 16429, 16336, 16206, 16171, 16094, 15921, 15821, 15809, 15652, 15532, 15464, 15414, 15348, 15326, 15275

In [ ]:
length=[]
for i in range(len(normalized_cleaned)): #Okt로 토큰화한 데이터
    length.append(len(normalized_cleaned[i]))
average=sum(length)/len(length)
print(average) # 966.4137025543089
print(max(length)) # 114063
print(min(length)) # 0
# 0이 아닌 최소 길이 찾기
min_length = min([l for l in length if l > 0])  # 0보다 큰 길이들 중에서 최소값 찾기
print(min_length) # 1

# 내림차순으로 정렬
sorted_length = sorted(length, reverse=True)

print("내림차순 정렬된 리스트:", sorted_length)
# 내림차순 정렬된 리스트: [114063, 86525, 70067, 61648, 54378, 49109, 46225,

현재 상태로 데이터를 넣으면 너무 길어서 안되고 okt로 토큰화한것도 안좋다.
KoBERT로 넣지 못하면 이전 K-Means 군집화로 구한 군집에따라 라벨링하면 된다.